In [7]:
library(ggplot2)
library(readr)
library(dplyr)
library(lubridate)
library(tidyr)
library(readxl)

In [6]:
install.packages("readxl")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rematch’, ‘cellranger’, ‘progress’




In [10]:
system("git -C ../COVID-19/ pull")

In [32]:
system("wget https://www2.census.gov/programs-surveys/popest/tables/2010-2019/state/totals/nst-est2019-01.xlsx")

In [24]:
census_xl <- readxl::read_xlsx('./nst-est2019-01.xlsx', skip = 3)

New names:
* `` -> ...1



In [36]:
census_xl %>% head()

...1,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
United States,308745538,308758105,309321666,311556874,313830990,315993715,318301008,320635163,322941311,324985539,326687501,328239523
Northeast,55317240,55318443,55380134,55604223,55775216,55901806,56006011,56034684,56042330,56059240,56046620,55982803
Midwest,66927001,66929725,66974416,67157800,67336743,67560379,67745167,67860583,67987540,68126781,68236628,68329004
South,114555744,114563030,114866680,116006522,117241208,118364400,119624037,120997341,122351760,123542189,124569433,125580448
West,71945553,71946907,72100436,72788329,73477823,74167130,74925793,75742555,76559681,77257329,77834820,78347268
.Alabama,4779736,4780125,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185


In [37]:
census_xl %>%
select(1, "2019") %>% 
head()

...1,2019
<chr>,<dbl>
United States,328239523
Northeast,55982803
Midwest,68329004
South,125580448
West,78347268
.Alabama,4903185


In [51]:
census_df <- 
census_xl %>%
transmute(region = `...1` %>% stringr::str_replace("\\.", ""),
          pop=`2019`) %>%
filter(!is.na(pop)) %>%
mutate(region = ifelse(region %in% state.name, 
                          setNames(state.abb, state.name)[region],
                          region),
          pop)

In [52]:
census_df %>% head()

region,pop
<chr>,<dbl>
United States,328239523
Northeast,55982803
Midwest,68329004
South,125580448
West,78347268
AL,4903185


In [53]:
write_csv(census_df, path = './census.csv')